In [63]:
import os
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils.notebook import display_scrollable_dataframe,plot_sailency
from data_loaders import CUB_extnded_dataset
from models import get_inception_transform
from IPython.display import display

from sailency import get_saliency_maps,saliency_score_part

%reload_ext autoreload
%autoreload 2

# Data analysis 
On the use of Majority voting on the concetps. First the data is loaded. 

In [64]:
# Settings for the experiment your running

# Settings for loading the dataset, make sure its the same as the one used to train the model. 
model_folder_MV = r"models\Sequential_Basemodel3"
model_folder_NoMV = r"models\Sequential_Basemodel2"


use_majority_voting_MV = True
use_majority_voting_NoMV = False

# You can find the settings in the .hydra folders config.yaml 
data_config_MV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_MV,
                'min_class_count':10,
                'return_visibility':True}


# You can find the settings in the .hydra folders config.yaml 
data_config_NoMV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_NoMV,
                'min_class_count':10,
                'return_visibility':True}

#Load models make sure the model is trained with the same settings as the data loader

X_to_C_path = os.path.join(model_folder_MV,"best_XtoC_model.pth")
C_to_Y_path = os.path.join(model_folder_MV,"best_CtoY_model.pth")

ModelXtoC_MV = torch.load(X_to_C_path,map_location=torch.device('cpu'))
ModelCtoY_MV = torch.load(C_to_Y_path,map_location=torch.device('cpu'))

X_to_C_path = os.path.join(model_folder_NoMV,"best_XtoC_model.pth")
C_to_Y_path = os.path.join(model_folder_NoMV,"best_CtoY_model.pth")

ModelXtoC_NoMV = torch.load(X_to_C_path,map_location=torch.device('cpu'))
ModelCtoY_NoMV = torch.load(C_to_Y_path,map_location=torch.device('cpu'))

C:\Users\AnneKjerstineDesler\AppData\Local\Temp\ipykernel_796\3203232163.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ModelXtoC_MV = torch.load(X_to_C_path,map_locat

In [65]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

112 312


In [66]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts and visibility
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts and visibility
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [ ]:
# get accuracy of both the XtoC and CtoY models
#end_idx = 100
end_idx = len(data_MV_val)
acc_concepts = 0
acc_class_true_concepts = 0
acc_class = 0
acc_class_threshold = 0
ModelXtoC = ModelXtoC_NoMV
ModelCtoY = ModelCtoY_NoMV
n_concepts_val = n_concepts_NoMV_val
for i, item in enumerate(tqdm(data_NoMV_val)):
    if i == end_idx:
        break
    img= item[0].unsqueeze(0)
    true_concepts = item[1][0].unsqueeze(0)
    true_classes = item[2].unsqueeze(0)
    pred_concepts = ModelXtoC(img)
    pred_classes = ModelCtoY(pred_concepts)
    pred_classes_true_concepts = ModelCtoY(true_concepts)

    # apply threshold of 0.5
    pred_concepts[pred_concepts < 0.5] = 0
    pred_concepts[pred_concepts >= 0.5] = 1

    pred_classes_threshold = ModelCtoY(pred_concepts)

    acc_concepts += torch.sum(pred_concepts == true_concepts).item()/n_concepts_val

    acc_class += (torch.argmax(pred_classes,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_true_concepts += (torch.argmax(pred_classes_true_concepts,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_threshold += (torch.argmax(pred_classes_threshold,dim=1) == torch.argmax(true_classes,dim=1)).item()

print(acc_concepts/end_idx)
print(acc_class/end_idx)
print(acc_class_true_concepts/end_idx)
print(acc_class_threshold/end_idx)

acc_concepts_NoMV_val_ours = acc_concepts/end_idx
acc_class_NoMV_val_ours = acc_class/end_idx
acc_class_true_concepts_NoMV_val_ours = acc_class_true_concepts/end_idx
acc_class_threshold_NoMV_val_ours = acc_class_threshold/end_idx


100%|██████████| 1198/1198 [15:35<00:00,  1.28it/s]

0.7942243482727624
0.15191986644407346
0.39565943238731216
0.14357262103505844


In [68]:
# get accuracy of both the XtoC and CtoY models
#end_idx = 100
end_idx = len(data_MV_val)
acc_concepts = 0
acc_class_true_concepts = 0
acc_class = 0
acc_class_threshold = 0
ModelXtoC = ModelXtoC_MV
ModelCtoY = ModelCtoY_MV
n_concepts_val = n_concepts_MV_val
for i, item in enumerate(tqdm(data_MV_val)):
    if i == end_idx:
        break
    img= item[0].unsqueeze(0)
    true_concepts = item[1].unsqueeze(0)
    true_classes = item[2].unsqueeze(0)
    pred_concepts = ModelXtoC(img)
    pred_classes = ModelCtoY(pred_concepts)
    pred_classes_true_concepts = ModelCtoY(true_concepts)

    # apply threshold of 0.5
    pred_concepts[pred_concepts < 0.5] = 0
    pred_concepts[pred_concepts >= 0.5] = 1

    pred_classes_threshold = ModelCtoY(pred_concepts)

    acc_concepts += torch.sum(pred_concepts == true_concepts).item()/n_concepts_val

    acc_class += (torch.argmax(pred_classes,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_true_concepts += (torch.argmax(pred_classes_true_concepts,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_threshold += (torch.argmax(pred_classes_threshold,dim=1) == torch.argmax(true_classes,dim=1)).item()


print(acc_concepts/end_idx)
print(acc_class/end_idx)
print(acc_class_true_concepts/end_idx)
print(acc_class_threshold/end_idx)

acc_concepts_MV_val_ours = acc_concepts/end_idx
acc_class_MV_val_ours = acc_class/end_idx
acc_class_true_concepts_MV_val_ours = acc_class_true_concepts/end_idx
acc_class_threshold_MV_val_ours = acc_class_threshold/end_idx


100%|██████████| 1198/1198 [17:51<00:00,  1.12it/s]

0.7952838063439077
0.5492487479131887
0.2287145242070117
0.5509181969949917


In [69]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, andreas=True)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, andreas=True)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, andreas=True)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, andreas=True)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

112 312


In [70]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [71]:
# get accuracy of both the XtoC and CtoY models
#end_idx = 100
end_idx = len(data_MV_val)
acc_concepts = 0
acc_class_true_concepts = 0
acc_class = 0
acc_class_threshold = 0
ModelXtoC = ModelXtoC_NoMV
ModelCtoY = ModelCtoY_NoMV
n_concepts_val = n_concepts_NoMV_val
for i, item in enumerate(tqdm(data_NoMV_val)):
    if i == end_idx:
        break
    img= item[0].unsqueeze(0)
    true_concepts = item[1][0].unsqueeze(0)
    true_classes = item[2].unsqueeze(0)
    pred_concepts = ModelXtoC(img)
    pred_classes = ModelCtoY(pred_concepts)
    pred_classes_true_concepts = ModelCtoY(true_concepts)

    # apply threshold of 0.5
    pred_concepts[pred_concepts < 0.5] = 0
    pred_concepts[pred_concepts >= 0.5] = 1

    pred_classes_threshold = ModelCtoY(pred_concepts)

    acc_concepts += torch.sum(pred_concepts == true_concepts).item()/n_concepts_val

    acc_class += (torch.argmax(pred_classes,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_true_concepts += (torch.argmax(pred_classes_true_concepts,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_threshold += (torch.argmax(pred_classes_threshold,dim=1) == torch.argmax(true_classes,dim=1)).item()


print(acc_concepts/end_idx)
print(acc_class/end_idx)
print(acc_class_true_concepts/end_idx)
print(acc_class_threshold/end_idx)

acc_concepts_NoMV_val_andreas = acc_concepts/end_idx
acc_class_NoMV_val_andreas = acc_class/end_idx
acc_class_true_concepts_NoMV_val_andreas = acc_class_true_concepts/end_idx
acc_class_threshold_NoMV_val_andreas = acc_class_threshold/end_idx



100%|██████████| 1198/1198 [14:33<00:00,  1.37it/s]

0.7942243482727624
0.15191986644407346
0.39565943238731216
0.14357262103505844


In [72]:
# get accuracy of both the XtoC and CtoY models
#end_idx = 100
end_idx = len(data_MV_val)
acc_concepts = 0
acc_class_true_concepts = 0
acc_class = 0
acc_class_threshold = 0
ModelXtoC = ModelXtoC_MV
ModelCtoY = ModelCtoY_MV
n_concepts_val = n_concepts_MV_val
for i, item in enumerate(tqdm(data_MV_val)):
    if i == end_idx:
        break
    img= item[0].unsqueeze(0)
    true_concepts = item[1].unsqueeze(0)
    true_classes = item[2].unsqueeze(0)
    pred_concepts = ModelXtoC(img)
    pred_classes = ModelCtoY(pred_concepts)
    pred_classes_true_concepts = ModelCtoY(true_concepts)

    # apply threshold of 0.5
    pred_concepts[pred_concepts < 0.5] = 0
    pred_concepts[pred_concepts >= 0.5] = 1

    pred_classes_threshold = ModelCtoY(pred_concepts)

    acc_concepts += torch.sum(pred_concepts == true_concepts).item()/n_concepts_val

    acc_class += (torch.argmax(pred_classes,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_true_concepts += (torch.argmax(pred_classes_true_concepts,dim=1) == torch.argmax(true_classes,dim=1)).item()
    acc_class_threshold += (torch.argmax(pred_classes_threshold,dim=1) == torch.argmax(true_classes,dim=1)).item()


print(acc_concepts/end_idx)
print(acc_class/end_idx)
print(acc_class_true_concepts/end_idx)
print(acc_class_threshold/end_idx)

acc_concepts_MV_val_andreas = acc_concepts/end_idx
acc_class_MV_val_andreas = acc_class/end_idx
acc_class_true_concepts_MV_val_andreas = acc_class_true_concepts/end_idx
acc_class_threshold_MV_val_andreas = acc_class_threshold/end_idx



100%|██████████| 1198/1198 [13:28<00:00,  1.48it/s]

0.9171312306224672
0.5492487479131887
0.9691151919866444
0.5509181969949917


In [73]:
print('Our dataloader')
print('NoMV')
print('concepts accuracy:', acc_concepts_NoMV_val_ours)
print('class accuracy with predicted concept probabilities:', acc_class_NoMV_val_ours)
print('class accuracy with 0.5 thresholded concepts:', acc_class_threshold_NoMV_val_ours)
print('class accuracy with true concepts as input:', acc_class_true_concepts_NoMV_val_ours)

print('MV')
print('concepts accuracy:', acc_concepts_MV_val_ours)
print('class accuracy with predicted concept probabilities:', acc_class_MV_val_ours)
print('class accuracy with 0.5 thresholded concepts:', acc_class_threshold_MV_val_ours)
print('class accuracy with true concepts as input:', acc_class_true_concepts_MV_val_ours)

print('Andreas dataloader')
print('NoMV')
print('concepts accuracy:', acc_concepts_NoMV_val_andreas)
print('class accuracy with predicted concept probabilities:', acc_class_NoMV_val_andreas)
print('class accuracy with 0.5 thresholded concepts:', acc_class_threshold_NoMV_val_andreas)
print('class accuracy with true concepts as input:', acc_class_true_concepts_NoMV_val_andreas)

print('MV')
print('concepts accuracy:', acc_concepts_MV_val_andreas)
print('class accuracy with predicted concept probabilities:', acc_class_MV_val_andreas)
print('class accuracy with 0.5 thresholded concepts:', acc_class_threshold_MV_val_andreas)
print('class accuracy with true concepts as input:', acc_class_true_concepts_MV_val_andreas)



Our dataloader
NoMV
concepts accuracy: 0.7942243482727624
class accuracy with predicted concept probabilities: 0.15191986644407346
class accuracy with 0.5 thresholded concepts: 0.14357262103505844
class accuracy with true concepts as input: 0.39565943238731216
MV
concepts accuracy: 0.7952838063439077
class accuracy with predicted concept probabilities: 0.5492487479131887
class accuracy with 0.5 thresholded concepts: 0.5509181969949917
class accuracy with true concepts as input: 0.2287145242070117
Andreas dataloader
NoMV
concepts accuracy: 0.7942243482727624
class accuracy with predicted concept probabilities: 0.15191986644407346
class accuracy with 0.5 thresholded concepts: 0.14357262103505844
class accuracy with true concepts as input: 0.39565943238731216
MV
concepts accuracy: 0.9171312306224672
class accuracy with predicted concept probabilities: 0.5492487479131887
class accuracy with 0.5 thresholded concepts: 0.5509181969949917
class accuracy with true concepts as input: 0.969115191

Very interesting and proves exactly why andreases code i completely wrong!!!!

The concept accuracies for the NoMV model are naturally exactly the same for our dataset and Andreas' dataset (0.795), since our dataset and Andreas' dataset are the same when no MV is applied to the validation labels. This is the concept accuracy that Andreas has used and visualized in the file concept_train_plot for basemodel 2 and should be the concept accuracy of the model with the lowest validation loss. It seems a bit lower that what is visualized in the plot, but maybe the lowest loss does not correspont to the highest accuracy.

The concept accuracy for the MV model on our dataset (0.794) is very similiar to the concept accuracy for the NoMV model and indicates that the models learn more or less the same concepts no matter if MV is used or not.

The concept accuracy for the MV model on Andreas' dataset is very high (0.917) as could be expected, since MV has been applied to the validation labels and the validation labels is thus much more similar to the training labels. This is the concept accuracy that Andreas has used and visualized in the file concept_train_plot for basemodel 3 and should be the concept accuracy of the model with the lowest validation loss. It seems a bit lower that what is visualized in the plot, but maybe the lowest loss does not correspont to the highest accuracy.

I calculated the class accuracies for when the model that takes in the predicted concept probabilites, for the model that takes in the thresholded concepts and for the model that takes in the the true concepts (this last one is what Andreas has done, when he trained the model).

Again all the class accuracies for the NoMV model are naturally exactly the same for our dataset and Andreas' dataset, since our dataset and Andreas' dataset are the same when no MV is applied to the validation labels.

Natually the class accuracies for the MV model are also the same when using our data and Andreas' data, when inputting the predticted concept probabilities/thresholded concepts, since the it is the same model used to obtain the predictions, resulting in the same outputs and the class labels are the same for the two datasets.


In all cases it doesn't make much of a difference when inputing the predicted probabilities or the thresholded concepts.


The interesting result are when looking at the class accuracy when taking in the true concepts, as Andreas' has done when he trained the models.

This class accuracy is reduced from 0.969 when using Andreas' dataset to 0.228 when using our dataset. (This shows how stupid his idea is and how bad the model acctually is) The number 0.969 should be what can be observed in the file class_train_plot.png for basemodel 3 which fits pretty well. What we acctually should have seen is the number 0.228.




In general from looking at the accuracies when the models are applied on our dataset, which we deem more correct and the only results that are actually fair to make any conclusion about, we see that the X_to_C_models perform very similarly no matter if MV is applied of not. Meaning that the NoMV model is able to learn the variety in each class.

When inputing the predicted concepts into the C_to_Y_models the MV model performs much better than the NoMV model. This could just be because of lack of hyperparameter tuning for the NoMV model, but it could also be because, the MV X_to_C_model has just learned to recognise each class and then output the MV concepts. And since the MV model was trained on MV concepts, it should be perform almost perfectly when the correct MV concepts are passed to the C_to_Y_model. 

If we assume that the true concept is acctually passed to the C_to_Y_model, as Andreas has done, the NoMV C_to_Y_model acctually performs better than the MV C_to_Y_model when using our data. This makes sense again since the MV C_to_Y_model was trained on the MV concepts and thus has only learn to recognise the MV concepts of each class and not the acctual concepts and it is thus not able to make predictions on data where MV has not been applied. The NoMV C_to_Y_model performs much better here because the validation data is more similar to the NoMV training data.

